### Otras Técnicas de RAG

#### Configuración

In [3]:
import os
from dotenv import load_dotenv, find_dotenv 
load_dotenv(find_dotenv())
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
! pip install wikipedia

In [ ]:
! pip install llama_index

#### Simple RAG

In [4]:
from llama_index.readers.wikipedia import WikipediaReader

reader = WikipediaReader()
pages = ['Jenna Ortega', 'Beetlejuice Beetlejuice', 'Winona Ryder']
documents = reader.load_data(pages=pages, auto_suggest=False, redirect = False)

In [5]:
documents

[Document(id_='50229776', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Jenna Marie Ortega (born September 27, 2002) is an American actress. She began her career as a child and received recognition for her role as a younger version of Jane in The CW comedy-drama series Jane the Virgin (2014–2019). She then won an Imagen Award for her leading role as Harley Diaz in the Disney Channel series Stuck in the Middle (2016–2018). She played Ellie Alves in the thriller series You (2019) and starred in the family film Yes Day (2021), both for Netflix.\nOrtega received praise for her performance as a traumatized high school student in the drama film The Fallout (2021). She gained wide recognition for portraying Wednesday Addams in the Netflix horror-comedy series Wednesday (2022–present), for which she received nominations at the Golden Globe, Primetime Emmy, and Screen Actors Guild Awards. She also starred in the slasher film

In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex

# Inicializa modelos
gpt= OpenAI( )
embed_model = OpenAIEmbedding()

# Transforma los chunks en vectores
index = VectorStoreIndex.from_documents(documents)
# Obtiene los embeddings similares
retriever = index.as_retriever(similarity_top_k=3)

In [7]:
from llama_index.core.prompts import PromptTemplate

# Construye una plantilla de prompt para solo proporcionar respuestas basadas en los documentos cargados 
template = (
"We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
    "Don't give an answer unless it is supported by the context above.\n"
)

qa_template = PromptTemplate(template)

In [8]:
# Crea un prompt para el modelo
question = "What is the plot of the movie in which Winona Ryder acted alongside Jenna Ortega?"

# Recupera el contexto
contexts = retriever.retrieve(question)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

# Genera la respuesta 
response = gpt.complete(prompt)
print(str(response))

The plot of the movie in which Winona Ryder acted alongside Jenna Ortega is not provided in the context above.


In [9]:
# Crea un prompt para el modelo
question = "Compare the families of Winona Ryder and Jenna Ortega"
# Recupera el contexto para el modelo
contexts = retriever.retrieve(question)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)
# Genera la respuesta
response = gpt.complete(prompt)
print(str(response))

Based on the context provided, Winona Ryder's family background is described in detail, including her parents, siblings, and family friends. It is mentioned that her father is of Ashkenazi Jewish descent and her mother is a Buddhist. Winona has a younger brother, Urie, and two older half-siblings from her mother's prior marriage. Her family friends include Timothy Leary, Allen Ginsberg, Lawrence Ferlinghetti, and Philip K. Dick. Additionally, it is noted that Winona's family lived on a commune in California when she was a child.

On the other hand, there is no information provided about Jenna Ortega's family background in the context above. Therefore, it is not possible to compare the families of Winona Ryder and Jenna Ortega based on the information given.


#### Updating Chunk Size

In [10]:
from llama_index.core import Settings

# modifica los valores predeterminados de tamaño de fragmento y superposición de fragmentos
Settings.chunk_size = 512
Settings.chunk_overlap = 50
Settings.embed_model = embed_model

In [11]:
# construye el índice
index = VectorStoreIndex.from_documents(
    documents,
)
# crea un motor de consulta
query_engine = index.as_query_engine(similarity_top_k=6)

In [12]:
# Genera una respuesta
response = query_engine.query("What is the plot of the movie in which Winona Ryder acted alongside Jenna Ortega?")
print(response)

The plot of the movie in which Winona Ryder acted alongside Jenna Ortega revolves around a woman who is haunted by her past and must confront terrifying forces in order to save her family.


#### Multi-step query engine

In [ ]:
# Multi-step query engine
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform
)
from llama_index.core.query_engine import MultiStepQueryEngine

step_decompose_transform_gpt3 = StepDecomposeQueryTransform(gpt, verbose=True)
index_summary = "Breaks down the initial query"

# Devuelve el query engine
multi_step_query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
    index_summary=index_summary
)

response = multi_step_query_engine.query("Compare the families of Jenna Ortega and Winona Ryder")

> Current query: Compare the families of Jenna Ortega and Winona Ryder
> New query: What are some key similarities and differences in the family backgrounds of Jenna Ortega and Winona Ryder?
> Current query: Compare the families of Jenna Ortega and Winona Ryder
> New query: What are the specific professions of Jenna Ortega's parents and Winona Ryder's parents that have influenced their acting careers?
> Current query: Compare the families of Jenna Ortega and Winona Ryder
> New query: How have the professions of Jenna Ortega's parents and Winona Ryder's parents influenced their acting careers?


In [19]:
response.response

"Both Jenna Ortega and Winona Ryder come from families with diverse professional backgrounds. Ortega's parents have worked in law enforcement and healthcare, while Ryder's parents have careers in writing, publishing, editing, and archiving. Despite these differences, both actresses have had supportive families that have played a role in their respective acting careers."